In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
text = """
artificial intelligence is transforming modern society
it is used in healthcare finance education and transportation
machine learning allows systems to improve automatically with experience
data plays a critical role in training intelligent systems
large datasets help models learn complex patterns
deep learning uses multi layer neural networks
neural networks are inspired by biological neurons
each neuron processes input and produces an output
training a neural network requires optimization techniques
gradient descent minimizes the loss function
"""

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_Words=len(tokenizer.word_index)+1

In [ ]:
input_sequences=[]
for line in text.split("\n"):
  token_list=tokenizer.texts_to_sequences([line])[0]

  for i in range(1,len(token_list)):
    n_gram_seq=token_list[:i+1]
    input_sequences.append(n_gram_seq)

max_seq_len=max([len(x) for x in input_sequences])
input_sequences=pad_sequences(input_sequences,maxlen=max_seq_len,padding='pre')

X=input_sequences[:,:-1]
Y=input_sequences[:,-1]

Y=tf.keras.utils.to_categorical(Y,num_classes=total_Words)


In [ ]:
model=Sequential([
    Embedding(total_Words,64,input_length=max_seq_len-1),
    LSTM(128),
    Dense(total_Words,activation='softmax')

])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.fit(X,Y,epochs=200,verbose=1)

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.0116 - loss: 4.1905  
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0812 - loss: 4.1824    
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0582 - loss: 4.1755
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0542 - loss: 4.1693
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0658 - loss: 4.1631
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0736 - loss: 4.1572
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1240 - loss: 4.1512
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0892 - loss: 4.1414
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0814 - loss: 4.1273
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0620 - loss: 4.1151
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0542 - loss: 4.1051
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.054

In [ ]:
def generate_text(seed_text,next_words=10):
  for _ in range(next_words):
    token_list=tokenizer.texts_to_sequences([seed_text])[0]
    token_list=pad_sequences([token_list],maxlen=max_seq_len-1,padding='pre')

    predicted=np.argmax(model.predict(token_list),axis=-1)[0]

    for word,index in tokenizer.word_index.items():
      if index==predicted:
        seed_text+=" "+word
        break
  return seed_text

print(generate_text("artificial intelligence"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
artificial intelligence is transforming modern society society and transportation transportation transportation output


Transformer Text Generator

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Embedding,LayerNormalization,MultiHeadAttention
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self,max_len,d_model):
    super().__init__()
    self.pos_encoding=self.positional_encoding(max_len,d_model)

  def positional_encoding(self,position,d_model):
    angles=np.arange(position)[:,np.newaxis]/np.power(10000,(2*(np.arange(d_model)[np.newaxis,:]//2))/np.float32(d_model))
    angles[:,0::2]=np.sin(angles[:,0::2])
    angles[:,1::2]=np.cos(angles[:,1::2])
    return tf.cast(angles[np.newaxis,...],dtype=tf.float32)


  def call(self,inputs):
    return inputs+self.pos_encoding[:,:tf.shape(inputs)[1],:]

class TransformerBlock(tf.keras.layers.Layer):
  def __init__(self,embed_dim,num_heads,ff_dim):
    super().__init__()
    self.att=MultiHeadAttention(num_heads=num_heads,key_dim=embed_dim)
    self.ffn=tf.keras.Sequential([
        Dense(ff_dim,activation="relu"),
        Dense(embed_dim),
    ])

    self.layernorm1=LayerNormalization()
    self.layernorm2=LayerNormalization()

  def call(self,inputs):
    attn_output=self.att(inputs,inputs)
    out1=self.layernorm1(inputs+attn_output)
    ffn_output=self.ffn(out1)
    return self.layernorm2(out1+ffn_output)

vocab_size=total_Words
max_len=max_seq_len-1
embed_dim=64

inputs=tf.keras.Input(shape=(max_len,))
x=Embedding(vocab_size,embed_dim)(inputs)
x=PositionalEncoding(max_len,embed_dim)(x)
x=TransformerBlock(embed_dim,2,128)(x)
x=tf.keras.layers.GlobalAveragePooling1D()(x)
outputs=Dense(vocab_size,activation="softmax")(x)

model=Model(inputs,outputs)
model.compile(loss="categorical_crossentropy",optimizer="adam")

model.fit(X,Y,epochs=100)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 4.4679
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 4.2700
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 4.3028
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 4.2772
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 4.2400
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 4.2139
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 4.1767
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4.1593
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 4.1767
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4.1896
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4.1543
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4.2161
Epoch 13/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4.1975
Epoch 14/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4.1529
Epoch 15/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4.1534
Epoch 16/100
3/3 ━━